In [1]:
# # pandas와 openpyxl 라이브러리 다운로드 후, import 해준다.
# import pandas as pd

# # 원하는 엑셀 파일을 pd.read_excel('') 함수를 이용해 데이터프레임으로 만들어준다.
# df1 = pd.read_excel('game_1_eng.xlsx')
# df2 = pd.read_excel('game_2_eng.xlsx')

# # merge 함수를 이용해 합쳐준다. left는 df1 기준으로 df2가 붙는다는 뜻.
# df3 = pd.merge(df1, df2)

# # 해당 데이터프레임을 엑셀로 저장해준다.
# df3.to_excel('test_merge.xlsx', index=False)

In [15]:
import urllib.request
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import re
import time
import nltk

from PIL import Image
from io import BytesIO
from nltk.tokenize import RegexpTokenizer
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from gensim.models.word2vec import Word2Vec

from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity

# Download stopwords
nltk.download('stopwords')

# Load data
df = pd.read_excel("gamedata_eng.xlsx")
print('Total number of documents:', len(df))

# Preview the first 5 rows
df.head()

# Data cleaning functions
def _removeNonAscii(s):
    return "".join(i for i in str(s) if  ord(i) < 128)

def make_lower_case(text):
    return text.lower()

def remove_stop_words(text):
    text = text.split()
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops]
    text = " ".join(text)
    return text

def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

def remove_punctuation(text):
    tokenizer = RegexpTokenizer(r'[a-zA-Z]+')
    text = tokenizer.tokenize(text)
    text = " ".join(text)
    return text


# # Data cleaning
# df['cleaned'] = df['about_the_game'].apply(_removeNonAscii)
# df['cleaned'] = df['cleaned'].apply(make_lower_case)
# df['cleaned'] = df['cleaned'].apply(remove_stop_words)
# df['cleaned'] = df['cleaned'].apply(remove_punctuation)
# df['cleaned'] = df['cleaned'].apply(remove_html)

# # Remove empty rows
# df['cleaned'].replace('', np.nan, inplace=True)
# df = df[df['cleaned'].notna()]
# print('Total number of documents after cleaning:', len(df))

# # Build Word2Vec model
# corpus = [words.split() for words in df['cleaned']]
# word2vec_model = Word2Vec(size=300, window=5, min_count=2, workers=-1) # make model
# word2vec_model.build_vocab(corpus) # 
# word2vec_model.intersect_word2vec_format('GoogleNews-vectors-negative300.bin.gz', lockf=1.0, binary=True) # pre-trained data
# word2vec_model.train(corpus, total_examples=word2vec_model.corpus_count, epochs=15) 

# # Get document vectors
# def get_document_vectors(document_list, model):
#     document_embedding_list = []
    
#     for line in document_list:
#         doc2vec = None
#         count = 0
#         for word in line.split():
#             if word in model.wv.vocab:
#                 count += 1
#                 if doc2vec is None:
#                     doc2vec = model[word]
#                 else:
#                     doc2vec = doc2vec + model[word]
#         if doc2vec is not None:
#             doc2vec = doc2vec / count
#             document_embedding_list.append(doc2vec)
    
#     return document_embedding_list

# document_embedding_list = get_document_vectors(df['cleaned'], word2vec_model)
# print('Number of document vectors:', len(document_embedding_list))
# print(document_embedding_list)

# # Calculate cosine similarity matrix
# cosine_similarities = cosine_similarity(document_embedding_list, document_embedding_list)
# print('Size of cosine similarity matrix:', cosine_similarities.shape)

# def get_movie_indices(name, df, cosine_similarities):
#     # Get the index of the movie based on its name
#     indices = pd.Series(df.index, index=df['name']).drop_duplicates()
#     idx = indices[name]

#     # Get the indices of the most similar movies based on the cosine similarities
#     sim_scores = list(enumerate(cosine_similarities[idx]))
#     sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
#     sim_scores = sim_scores[1:6]
#     movie_indices = [i[0] for i in sim_scores]

#     return movie_indices

# def show_recommendations(name, df, cosine_similarities):
#     movies = df[['name', 'image']]

#     # Get the indices of the recommended movies
#     movie_indices = get_movie_indices(name, df, cosine_similarities)

#     # Get the recommended movies based on their indices
#     recommend = movies.iloc[movie_indices].reset_index(drop=True)

#     fig, axs = plt.subplots(1, 5, figsize=(20, 30))

#     # Display the images of the recommended movies
#     for index, row in recommend.iterrows():
#         try:
#             response = requests.get(row['image'])
#             img = Image.open(BytesIO(response.content))
#             axs[index].imshow(img)
#             axs[index].set_title(row['name'])
#         except:
#             continue
#     plt.show()



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\multicampus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Total number of documents: 57068


In [ ]:

# Data cleaning
df['cleaned'] = df['about_the_game'].apply(_removeNonAscii)
df['cleaned'] = df['cleaned'].apply(make_lower_case)
df['cleaned'] = df['cleaned'].apply(remove_stop_words)
df['cleaned'] = df['cleaned'].apply(remove_punctuation)
df['cleaned'] = df['cleaned'].apply(remove_html)

# Remove empty rows
df['cleaned'].replace('', np.nan, inplace=True)
df = df[df['cleaned'].notna()]
print('Total number of documents after cleaning:', len(df))

# Build Word2Vec model
corpus = [words.split() for words in df['cleaned']]
word2vec_model = Word2Vec(size=300, window=5, min_count=2, workers=-1) # make model
word2vec_model.build_vocab(corpus) # 
word2vec_model.intersect_word2vec_format('GoogleNews-vectors-negative300.bin.gz', lockf=1.0, binary=True) # pre-trained data
word2vec_model.train(corpus, total_examples=word2vec_model.corpus_count, epochs=15) 

# Get document vectors
def get_document_vectors(document_list, model):
    document_embedding_list = []
    
    for line in document_list:
        doc2vec = None
        count = 0
        for word in line.split():
            if word in model.wv.vocab:
                count += 1
                if doc2vec is None:
                    doc2vec = model[word]
                else:
                    doc2vec = doc2vec + model[word]
        if doc2vec is not None:
            doc2vec = doc2vec / count
            document_embedding_list.append(doc2vec)
    
    return document_embedding_list

document_embedding_list = get_document_vectors(df['cleaned'], word2vec_model)
print('Number of document vectors:', len(document_embedding_list))
print(document_embedding_list)

# Calculate cosine similarity matrix
cosine_similarities = cosine_similarity(document_embedding_list, document_embedding_list)
print('Size of cosine similarity matrix:', cosine_similarities.shape)

def get_movie_indices(name, df, cosine_similarities):
    # Get the index of the movie based on its name
    indices = pd.Series(df.index, index=df['name']).drop_duplicates()
    idx = indices[name]

    # Get the indices of the most similar movies based on the cosine similarities
    sim_scores = list(enumerate(cosine_similarities[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]
    movie_indices = [i[0] for i in sim_scores]

    return movie_indices

def show_recommendations(name, df, cosine_similarities):
    movies = df[['name', 'image']]

    # Get the indices of the recommended movies
    movie_indices = get_movie_indices(name, df, cosine_similarities)

    # Get the recommended movies based on their indices
    recommend = movies.iloc[movie_indices].reset_index(drop=True)

    fig, axs = plt.subplots(1, 5, figsize=(20, 30))

    # Display the images of the recommended movies
    for index, row in recommend.iterrows():
        try:
            response = requests.get(row['image'])
            img = Image.open(BytesIO(response.content))
            axs[index].imshow(img)
            axs[index].set_title(row['name'])
        except:
            continue
    plt.show()


Total number of documents after cleaning: 57051


In [ ]:
show_recommendations("ENEMIES", df, cosine_similarities)

In [ ]:
show_recommendations("Zombie Freaks", df, cosine_similarities)

In [ ]:
show_recommendations("The Hell Hives", df, cosine_similarities)

In [ ]:
show_recommendations("MAZE: A VR Adventure", df, cosine_similarities)

In [ ]:
show_recommendations("Woods of Death 2", df, cosine_similarities)